# Imports

In [ ]:
from __future__ import annotations
from typing import Tuple, List, Dict, Optional, Any

import numpy as np
import torch
from pathlib import Path
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset, DownloadMode

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import tqdm

from helper_utils.enum_keys import (
    FPKey,
    ModelKey,
    QuantStyle,
    MiscPrompts,
    Contexts,
    Texts
)

from PTQ.bitlinear_wrapper_class import BitLinear
from PTQ.apply_ptq import applyPTQ
from PTQ.olmo_act_fns import patch_olmo_mlp
import helper_utils.utils as utils
from helper_utils.models_loader import load_4bit_auto, load_8bit_auto
from mech_interp_utils.utils_main.src.transformer_utils import (
    logit_lens,
    activation_lens,
    dictionary_learning,
    chatbot_analysis
)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
torch.manual_seed(0)
original = torch.randn(512) * 0.5  # Original activations

def quantize_dequantize(tensor, scale_value):
    scale = max(scale_value, 1e-8)
    qmin, qmax = -127, 127
    tensor_int = (tensor / scale).round().clamp(qmin, qmax).to(torch.int8)
    tensor_dequant = tensor_int.float() * scale
    return tensor_int, tensor_dequant

# Quantize with different scales
_, dequant_1e2 = quantize_dequantize(original, 1e-2)
_, dequant_1e5 = quantize_dequantize(original, 1e-5)

# L2 distance
print("L2 Distance (scale=1e-2):", torch.norm(original - dequant_1e2).item())
print("L2 Distance (scale=1e-5):", torch.norm(original - dequant_1e5).item())

# Plot histograms + KDEs
plt.figure(figsize=(14, 6))
sns.histplot(original.numpy(),bel='Original', kde=True, stat="count", bins=50, color='black', alpha=0.5)
sns.histplot(dequant_1e2.numpy(), label='Dequant (scale=1e-2)', kde=True, stat="count", bins=50, color='red', alpha=0.5)
sns.histplot(dequant_1e5.numpy(), label='Dequant (scale=1e-5)', kde=True, stat="count", bins=50, color='blue', alpha=0.5)

plt.title("Histogram (Count) + KDE of Quantized vs Original Activations")
plt.xlabel("Activation Value")
plt.ylabel("Count")
plt.ylim(0, 40)  
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('Outputs/fake_tensor_quant_scale.png')
plt.show()


In [ ]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True

### Dataset for calibrating activations

In [ ]:
filepath = r'D:\ThesisData\wikitext'

destination_path = str(Path(filepath))
dataset = load_dataset(
    'wikitext', 'wikitext-103-raw-v1',
    split={
        'train': 'train[:30%]',
        'validation': 'validation[:10%]',
        'test': 'test[:10%]',
    },
    cache_dir=destination_path,
    download_mode=DownloadMode.REUSE_DATASET_IF_EXISTS,
    keep_in_memory=True
)

In [ ]:
train_texts = dataset['train']

In [ ]:
calibration_texts = [t for t in dataset['train']["text"] if isinstance(t, str) and t.strip()]
#calibration_texts = [t for t in sub_txts["text"] if isinstance(t, str) and t.strip()]

In [ ]:
sub_txts = train_texts.take(50)

### Params

In [ ]:
PARAMS:Dict = {
    'context': Contexts.C1.value,
    'prompt': MiscPrompts.Q11.value,
    'max_new_tokens': 250,
    'temperature': 0.8,
    'repetition_penalty': 1.1,
    'sample': True,
    'device': None
}

# Models and Tokenizer

In [ ]:
def load_test_model(model_path:str, dtype=torch.dtype) -> AutoModelForCausalLM:
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        return_dict=True,
        output_hidden_states=True,
        torch_dtype=dtype,
        low_cpu_mem_usage=True,
        local_files_only=True,
        use_safetensors=True,
        #trust_remote_code=True
    )

    return model

In [ ]:
hfbit1_tokenizer = AutoTokenizer.from_pretrained(FPKey.HFBIT1_TOKENIZER.value)

In [ ]:
hfbit1_fp32 = load_test_model(FPKey.HFBIT1_8B.value, dtype=torch.float32) # https://huggingface.co/HF1BitLLM/Llama3-8B-1.58-100B-tokens

In [ ]:
hfbit1_fp32

In [ ]:
llama8b_tokenizer = AutoTokenizer.from_pretrained(FPKey.LINSTRUCT_TOKENIZER.value)

In [ ]:
llama8b_fp32 = load_test_model(FPKey.LINSTRUCT_8B.value, dtype=torch.float32)

In [ ]:
llama8b_bnb4_fp16 = load_4bit_auto(
    KEY=ModelKey.LLINSTRUCT8B.value,
    hs=True,
    r_dict=True,
    precision=torch.float16,
    dmap='auto',
    sf=True,
    trust_remote=True
)

In [ ]:
llama8b_bnb4_fp16

In [ ]:
hfbit1_tokenizer.vocab_size

In [ ]:
llama8b_tokenizer.vocab_size

In [ ]:
txt = Texts.T1.value

In [ ]:
llama8b_bitnet_fp32 = applyPTQ(
    load_test_model(FPKey.LINSTRUCT_8B.value, dtype=torch.float32),
    tokenizer=llama8b_tokenizer,
    #calibration_input=None,
    #calibration_input=sub_txts['text'],
    calibration_input=Texts.T1.value,
    mode='1.58bit',
    safer_quant=True,
    model_half=False,
    quant_half=False,
    layers_to_quant_weights=QuantStyle.BITNET.value,
    layers_to_quant_activations=QuantStyle.BITNET.value,
    fragile_layers=False,
    act_quant=True,
    act_bits=8,
    debugging=True,
    plot_debugging=False,
    plot_quantization=False,
    freeze_modules=True,
    #save_model_path='Decoders/NousResearch/LlamaInstruct8B/Meta-Llama-3.1-8B-Instruct-1.58-ptsq'
    save_model_path=None
)

In [ ]:
llama8b_bitnet_fp32

In [ ]:
utils.print_model_weights(llama8b_bitnet_fp32)

### allenai/OLMo

In [ ]:
olmo1b_tokenizer = AutoTokenizer.from_pretrained(FPKey.OLMO1B_TOKENIZER.value)

In [ ]:
olmo2t_tokenizer = AutoTokenizer.from_pretrained(FPKey.OLMO7B2T_TOKENIZER.value)

In [ ]:
olmo7b_tokenizer = AutoTokenizer.from_pretrained(FPKey.OLMO7B_TOKENIZER.value)

In [ ]:
olmo1b_fp32 = load_test_model(FPKey.OLMO1B_FP.value, dtype=torch.float32)

In [ ]:
olmo2t_fp32 = load_test_model(FPKey.OLMO7B2T_FP.value, dtype=torch.float32)

In [ ]:
olmo7b_fp32 = load_test_model(FPKey.OLMO7B_FP.value, dtype=torch.float32)

In [ ]:
olmo1b_fp32

In [ ]:
olmo1b_fp32.config

In [ ]:
QuantStyle.BITNET.value

In [ ]:
Texts.T1.value

In [ ]:
olmo1b_bitnet_fp32_qlmhead = applyPTQ(
    load_test_model(FPKey.OLMO1B_FP.value, dtype=torch.float32),
    tokenizer=olmo1b_tokenizer,
    #calibration_input=None,
    #calibration_input=sub_txts['text'],
    calibration_input=Texts.T1.value,
    mode='1.58bit',
    safer_quant=True,
    q_lmhead=True,
    model_half=False,
    quant_half=False,
    layers_to_quant_weights=QuantStyle.BITNET.value,
    layers_to_quant_activations=QuantStyle.BITNET.value,
    fragile_layers=False,
    act_quant=True,
    act_bits=8,
    debugging=True,
    plot_debugging=False,
    plot_quantization=False,
    freeze_modules=True
)

In [ ]:
olmo1b_bitnet_fp32_flmhead = applyPTQ(
    load_test_model(FPKey.OLMO1B_FP.value, dtype=torch.float32),
    tokenizer=olmo1b_tokenizer,
    #calibration_input=None,
    #calibration_input=sub_txts['text'],
    calibration_input=Texts.T1.value,
    mode='1.58bit',
    safer_quant=True,
    q_lmhead=False,
    model_half=False,
    quant_half=False,
    layers_to_quant_weights=QuantStyle.BITNET.value,
    layers_to_quant_activations=QuantStyle.BITNET.value,
    fragile_layers=False,
    act_quant=True,
    act_bits=8,
    debugging=True,
    plot_debugging=False,
    plot_quantization=False,
    freeze_modules=True
)

In [ ]:
olmo2t_bitnet_fp32_qlmhead = applyPTQ(
    load_test_model(FPKey.OLMO7B2T_FP.value, dtype=torch.float32),
    tokenizer=olmo2t_tokenizer,
    #calibration_input=None,
    #calibration_input=sub_txts['text'],
    calibration_input=Texts.T1.value,
    mode='1.58bit',
    safer_quant=True,
    q_lmhead=True,
    model_half=False,
    quant_half=False,
    layers_to_quant_weights=QuantStyle.BITNET.value,
    layers_to_quant_activations=QuantStyle.BITNET.value,
    fragile_layers=False,
    act_quant=True,
    act_bits=8,
    debugging=True,
    plot_debugging=False,
    plot_quantization=False,
    freeze_modules=True
)

In [ ]:
olmo2t_bitnet_fp32_flmhead = applyPTQ(
    load_test_model(FPKey.OLMO7B2T_FP.value, dtype=torch.float32),
    tokenizer=olmo2t_tokenizer,
    #calibration_input=None,
    #calibration_input=sub_txts['text'],
    calibration_input=Texts.T1.value,
    mode='1.58bit',
    safer_quant=True,
    q_lmhead=False,
    model_half=False,
    quant_half=False,
    layers_to_quant_weights=QuantStyle.BITNET.value,
    layers_to_quant_activations=QuantStyle.BITNET.value,
    fragile_layers=False,
    act_quant=True,
    act_bits=8,
    debugging=True,
    plot_debugging=False,
    plot_quantization=False,
    freeze_modules=True
)

#### NousResearch/OLMo-Bitnet-1B

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/OLMo-Bitnet-1B", trust_remote_code=True)

In [ ]:
utils.save_tokenizer(tokenizer, 'Decoders/NousResearch/OLMoBitnet/OLMo-Bitnet-1B-tokenizer')

In [ ]:
model = AutoModelForCausalLM.from_pretrained("NousResearch/OLMo-Bitnet-1B", trust_remote_code=True)

In [ ]:
utils.save_model(model, 'Decoders/NousResearch/OLMoBitnet/OLMo-Bitnet-1B')

### NousResearch/DeepHermes

In [ ]:
dh3b_tokenizer = AutoTokenizer.from_pretrained(FPKey.TOKENIZER_3B.value)

In [ ]:
dh8b_tokenizer = AutoTokenizer.from_pretrained(FPKey.TOKENIZER_8B.value)

In [ ]:
dh3b_fp32 = load_test_model(FPKey.FP_3B.value, dtype=torch.float32)

In [ ]:
dh3b_bnb4_fp16 = load_4bit_auto(
    KEY=ModelKey.DEEP3B.value,
    hs=True,
    r_dict=True,
    precision=torch.float16,
    dmap='auto',
    sf=True,
    trust_remote=True
)

In [ ]:
dh3b_bnb4_fp16

In [ ]:
dh8b_fp32 = load_test_model(FPKey.FP_8B.value, dtype=torch.float32)

In [ ]:
dh8b_bnb4_fp16 = load_4bit_auto(
    KEY=ModelKey.DEEP8B.value,
    hs=True,
    r_dict=True,
    precision=torch.float16,
    dmap='auto',
    sf=True,
    trust_remote=True
)

In [ ]:
dh3b_bitnet_fp32_flmhead = applyPTQ(
    load_test_model(FPKey.FP_3B.value, dtype=torch.float32),
    tokenizer=dh3b_tokenizer,
    calibration_input=None,
    #calibration_input=sub_txts['text'],
    #calibration_input=Texts.T1.value,
    mode='1.58bit',
    safer_quant=True,
    q_lmhead=False,
    model_half=False,
    quant_half=False,
    layers_to_quant_weights=QuantStyle.BITNET.value,
    layers_to_quant_activations=QuantStyle.BITNET.value,
    fragile_layers=False,
    act_quant=False,
    act_bits=8,
    debugging=True,
    plot_debugging=False,
    plot_quantization=False,
    freeze_modules=True
)

In [ ]:
dh3b_bitnet_fp32_qlmhead = applyPTQ(
    load_test_model(FPKey.FP_3B.value, dtype=torch.float32),
    tokenizer=dh3b_tokenizer,
    #calibration_input=None,
    calibration_input=sub_txts['text'],
    #calibration_input=Texts.T1.value,
    mode='1.58bit',
    safer_quant=True,
    q_lmhead=True,
    model_half=False,
    quant_half=False,
    layers_to_quant_weights=QuantStyle.BITNET.value,
    layers_to_quant_activations=QuantStyle.BITNET.value,
    fragile_layers=False,
    act_quant=True,
    act_bits=8,
    debugging=True,
    plot_debugging=False,
    plot_quantization=False,
    freeze_modules=True
)

In [ ]:
dh3b_bitnet_fp32

In [ ]:
dh8b_bitnet_fp32 = applyPTQ(
    load_test_model(FPKey.FP_8B.value, dtype=torch.float32),
    tokenizer=dh8b_tokenizer,
    #calibration_input=None,
    #calibration_input=sub_txts['text'],
    calibration_input=txt,
    mode='1.58bit',
    safer_quant=True,
    model_half=False,
    quant_half=False,
    layers_to_quant_weights=QuantStyle.BITNET.value,
    layers_to_quant_activations=QuantStyle.BITNET.value,
    fragile_layers=False,
    act_quant=True,
    act_bits=8,
    debugging=True,
    plot_debugging=False,
    plot_quantization=False,
    freeze_modules=True
)

In [ ]:
dh8b_bitnet_fp32

# Activation Lens and Logit Lens

In [ ]:
MiscPrompts.Q2.value

In [ ]:
MiscPrompts.Q11.value

In [ ]:
MiscPrompts.Q12.value

In [ ]:
llama8b_fp32.cpu()

In [ ]:
logit_lens.plot_logit_lens_plotly(
    model=olmo1b_fp32,
    tokenizer=olmo1b_tokenizer,
    input_ids=MiscPrompts.Q2.value,
    start_ix=0, end_ix=15,
    topk=5,
    #save_fig_path=None,
    #save_fig_path='Outputs/LogitLens/DH3B/logits_3b_fp32_math.jpg',
    entropy=True,
)

In [ ]:
logit_lens.plot_logit_lens(
    model=dh3b_fp32,
    tokenizer=dh3b_tokenizer,
    input_ids=MiscPrompts.Q12.value,
    start_ix=0, end_ix=15,
    #save_fig_path=None,
    #save_fig_path='Outputs/LogitLens/LI8B/probs_hf1bit_fp32_qa_blockstep10.jpg',
    entropy=True,
    block_step=10,
)

In [ ]:
logit_lens.plot_logit_lens(
    model=dh3b_bitnet_fp32_qlmhead,
    tokenizer=dh3b_tokenizer,
    input_ids=MiscPrompts.Q12.value,
    start_ix=0, end_ix=15,
    #save_fig_path=None,
    #save_fig_path='Outputs/LogitLens/LI8B/probs_hf1bit_fp32_qa_blockstep10.jpg',
    entropy=True,
    block_step=10,
)

In [ ]:
logit_lens.plot_logit_lens(
    model=dh3b_bitnet_fp32_flmhead,
    tokenizer=dh3b_tokenizer,
    input_ids=MiscPrompts.Q12.value,
    start_ix=0, end_ix=15,
    #save_fig_path=None,
    save_fig_path='Outputs/LogitLens/DH3B/logits_flmhead_fp32_qa.jpg',
    #kl=True,
)

In [ ]:
logit_lens.plot_comparing_lens(
    models=(dh3b_fp32, dh3b_bitnet_fp32_qlmhead),
    tokenizer=dh3b_tokenizer,
    input_ids=MiscPrompts.Q12.value,
    start_ix=0, end_ix=15,
    #save_fig_path='Outputs/LogitLens/LI8B/nwd_llama_hf1bit_qa_blockstep10.jpg',
    save_fig_path=None,
    wasserstein=True,
    #top_down=False,
    block_step=10,
)

In [ ]:
logit_lens.plot_comparing_lens(
    models=(olmo1b_bitnet_fp32_flmhead, olmo1b_bitnet_fp32_qlmhead),
    tokenizer=olmo1b_tokenizer,
    input_ids=MiscPrompts.Q2.value,
    start_ix=0, end_ix=15,
    save_fig_path='Outputs/LogitLens/OLMo1B/nwd_flmhead_qlmhead_math.jpg',
    #save_fig_path=None,
    wasserstein=True,
    #top_down=False,
)

In [ ]:
logit_lens.plot_comparing_lens(
    models=(llama8b_fp32, hfbit1_fp32),
    tokenizer=hfbit1_tokenizer,
    input_ids=MiscPrompts.Q12.value,
    start_ix=0, end_ix=15,
    #save_fig_path='Outputs/LogitLens/LI8B/nwd_instruct_bitnet_fp32_qa.jpg',
    save_fig_path=None,
    wasserstein=True,
    #top_down=False,
)

In [ ]:
logit_lens.plot_comparing_lens(
    models=(llama8b_fp32, llama8b_bnb4_fp16),
    tokenizer=llama8b_tokenizer,
    input_ids=MiscPrompts.Q12.value,
    start_ix=0, end_ix=15,
    #save_fig_path='Outputs/LogitLens/DH3B/nwd_3bfp32_ptdq_math.jpg',
    save_fig_path=None,
    wasserstein=True,
    #top_down=False,
)

In [ ]:
logit_lens.plot_topk_lens(
    model=llama8b_bnb4_fp16,
    tokenizer=llama8b_tokenizer,
    input_ids=MiscPrompts.Q12.value,
    start_ix=0, end_ix=15,
    topk_n=5,
    save_fig_path='Outputs/LogitLens/LI8B/topk5logits_bnb4bit_qa.jpg'
    #save_fig_path=None,
    #top_down=False,
)

In [ ]:
logit_lens.plot_topk_lens(
    model=llama8b_bitnet_fp32,
    tokenizer=llama8b_tokenizer,
    input_ids=MiscPrompts.Q12.value,
    start_ix=0, end_ix=15,
    topk_n=5,
    save_fig_path='Outputs/LogitLens/LI8B/topk5logits_ptsq_fp32_math.jpg'
    #save_fig_path=None,
    #entropy=True,
    #top_down=False,
)

In [ ]:
activation_lens.plot_activation_lens(
    model=dh3b_bitnet_fp32_qlmhead,
    tokenizer=dh3b_tokenizer,
    input_ids=MiscPrompts.Q2.value,
    start_ix=0, end_ix=15,
    metric='norm',
    save_fig_path='Outputs/LogitLens/DH3B/actnorm_qlmhead_ptdq_math.jpg', #HUSK FOR BITNET hfb1 Q12 qa!
    #save_fig_path=None,
)

In [ ]:
activation_lens.plot_activation_lens(
    model=llama8b_bnb4_fp16,
    tokenizer=llama8b_tokenizer,
    input_ids=MiscPrompts.Q12.value,
    start_ix=0, end_ix=15,
    metric='norm',
    save_fig_path='Outputs/LogitLens/LI8B/actnorm_bnb4bit_fp16_qa.jpg',
    #save_fig_path=None,
)

In [ ]:
activation_lens.plot_comparing_act_lens(
    models=(llama8b_fp32, hfbit1_fp32),
    tokenizer=llama8b_tokenizer,
    input_ids=MiscPrompts.Q12.value,
    start_ix=0, end_ix=15,
    metric='norm',
    metric_name='l2',
    save_fig_path='Outputs/LogitLens/LI8B/actnorm_comparing_fp32_qa.jpg',
    #save_fig_path=None,
)

# Dictionary Learning: SAE

In [ ]:
dictionary_learning.plot_sae_tokens(
    model=dh3b_fp32,
    tokenizer=dh3b_tokenizer,
    inputs=PARAMS.get('prompt'),
    multi_tokens=False,
    do_log=False,
    target_layers=[5],
    vis_projection=None,
    log_path=None,
    log_name=None,
    fig_path=None
)

In [ ]:
dictionary_learning.plot_sae_tokens(
    model=dh3b_fp32,
    tokenizer=dh3b_tokenizer,
    inputs=PARAMS.get('prompt'),
    multi_tokens=True,
    do_log=False,
    target_layers=[5],
    vis_projection=None,
    log_path=None,
    log_name=None,
    fig_path=None
)

In [ ]:
dictionary_learning.plot_sae_tokens(
    model=dh3b_bitnet_fp32,
    tokenizer=dh3b_tokenizer,
    inputs=PARAMS.get('prompt'),
    multi_tokens=True,
    do_log=False,
    target_layers=[5],
    vis_projection=None,
    log_path=None,
    log_name=None,
    fig_path=None
)

In [ ]:
dictionary_learning.plot_sae_heatmap(
    model=hfbit1_fp32,
    tokenizer=hfbit1_tokenizer,
    inputs=Texts.T1.value,
    do_log=False,
    top_k=5,
    tokens_per_row=30,
    target_layers=[5,25,30],
    log_path=None,
    log_name=None,
    fig_path=None,
    deterministic_sae=False
)

In [ ]:
dictionary_learning.plot_sae_heatmap(
    model=llama8b_bnb4_fp16,
    tokenizer=llama8b_tokenizer,
    inputs=Texts.T1.value,
    do_log=False,
    top_k=5,
    tokens_per_row=30,
    target_layers=[5],
    log_path=None,
    log_name=None,
    fig_path=None
)

In [ ]:
dictionary_learning.plot_comparing_heatmap(
    models=(llama8b_fp32, hfbit1_fp32),
    tokenizer=llama8b_tokenizer,
    inputs=Texts.T1.value,
    top_k=5,
    tokens_per_row=30,
    target_layers=[25],
    fig_path=None,
    deterministic_sae=True
)

In [ ]:
dictionary_learning.plot_comparing_heatmap(
    models=(olmo1b_fp32, olmo1b_bitnet_fp32),
    tokenizer=olmo1b_tokenizer,
    inputs=Texts.T1.value,
    top_k=5,
    tokens_per_row=30,
    target_layers=[5],
    fig_path=None
)

# Deep Hermes Chatbot Analysis (template only)

In [ ]:
chat_dict = {
    #'dh.3b-llama.fp32': dh3b_fp32,
    'dh.3b-bnb4bit.fp16': dh3b_bnb4_fp16,
    #'dh.3b-1.58.ptdq': dh3b_bitnet_fp32, 
    #'dh.3b-1.58.ptsq': dh3b_bitnet_fp32,
    #'dh.8b-llama.fp32': dh8b_fp32,
    #'dh.8b-bnb4bit.fp16': dh8b_bnb4_fp16,
    #'dh.8b-1.58.ptdq': dh8b_bitnet_fp32,
    #'dh.8b-1.58.ptsq': dh8b_bitnet_fp32,
    #'llama.8b-instruct.fp32': llama8b_fp32,
    #'llama.8b-bnb4bit.fp16': llama8b_bnb4_fp16,
    #'llama.8b-1.58.fp32': hfbit1_fp32,
    #'llama.8b-1.58.ptdq': llama8b_bitnet_fp32,
    #'llama.8b-1.58.ptsq': llama8b_bitnet_fp32,
}

In [ ]:
PARAMS:Dict = {
    'context': Contexts.C1.value,
    'prompt': MiscPrompts.Q2.value,
    'max_new_tokens': 100,
    'temperature': 0.8,
    'repetition_penalty': 1.1,
    'sample': True,
    'device': None
}

In [ ]:
chatbot_analysis.run_chatbot_analysis(
    models=chat_dict,
    tokenizer=dh3b_tokenizer,
    deep_thinking=False,
    full_path='logs/chatbot_logs',
    deterministic_backend=False
)

In [ ]:
chatbot_analysis.plot_chatbot_analysis(
    json_logs='logs/math/chatbot_logs',
    parallel_plot=True,
    reference_file='logs/chatbot_logs/math/dh.8b-llama.fp32.json'
)

In [ ]:
chatbot_analysis.plot_chatbot_analysis(
    json_logs='logs/chatbot_logs/QA',
    parallel_plot=False,
    reference_file='logs/chatbot_logs/QA/llama.8b-instruct.fp32.json',
    title="Model Metrics ('What is y if y=2*2-4+(3*2)')"
)